<a href="https://colab.research.google.com/github/mssamoilenko/Data_enginWindow/blob/main/Data_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Згенеруйте дані на основі ковзаючих вікон(цільовий стовпчик `totals_transactionRevenue`)

In [ ]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/refs/heads/main/module3/data/data.csv",
                 index_col=0)

In [ ]:
df.

In [ ]:
df.head()

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device_browser,device_browserVersion,device_browserSize,...,trafficSource_keyword,trafficSource_adwordsClickInfo.criteriaParameters,trafficSource_referralPath,trafficSource_isTrueDirect,trafficSource_adContent,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.isVideoAd
0,Organic Search,20171016,5882839380479893026,Not Socially Engaged,1508198450,1,1508198450,Firefox,not available in demo dataset,not available in demo dataset,...,water bottle,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Referral,20171016,169064020318706586,Not Socially Engaged,1508176307,6,1508176307,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,not available in demo dataset,/a/google.com/transportation/mtv-services/bike...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Direct,20171016,6157754009880713732,Not Socially Engaged,1508201613,1,1508201613,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,not available in demo dataset,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
3,Organic Search,20171016,7805861212153847553,Not Socially Engaged,1508169851,1,1508169851,Chrome,not available in demo dataset,not available in demo dataset,...,(not provided),not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Organic Search,20171016,5130680452619315824,Not Socially Engaged,1508190552,1,1508190552,Chrome,not available in demo dataset,not available in demo dataset,...,(not provided),not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Завдання 1

Перетворіть стовпчик `date` у тип дати та виведіть гістограму. На її основі виберіть розміри вікон та пропуск

# Завдання 2

Отримайте дані з відповідних вікон

Формули для вікон по нумерації k=1,2,3,...

```bash
lower = df['date'].min() + (k-1)*window_X
upper = df['date'].min() + k*window_X
mask = (lower < df['date']) & (df['date'] < upper)
X = df[mask]
```

```bash
lower = df['date'].min() + k*window_X + gap
upper = df['date'].min() + k*window_X + gap + window_y
mask = (lower < df['date']) & (df['date'] < upper)
y = df[mask]
```

# Завдання 3

Створіть словник де ключ -- назва стовпчика, а значення -- список функцій:
* для числових - sum, mean, min, max
* для категоріальних - mode

Згрупуйте дані по користувачах(лише Х) та обрахуйте відповідні функції

# Завдання 4

Для даних з пізнішого вікна(у) обрахуйте суму `totals_transactionRevenue` для кожного користувача

# Завдання 5

Об'єднайте дані по id користувачів, якщо з'являються NaN, то замініть нулем(користувач не користувався системою)

# Завдання 6

Напишіть функцію `select_info(k)` яка містить код із завдань 2-5 та згенеровані дані для конкретного вікна

# Завдання 7

Якщо все зроблено правильно, наступний код має повернути таблицю по всіх вікнах

In [ ]:
full_data = pd.DataFrame()
num = 10 # кількість вікон, можете змінити

for k in range(1, num+1):
    data = select_info(k)
    full_data = pd.concat([full_data, data], axis=0)

full_data